In [1]:
from langchain_openai import ChatOpenAI
import os
import openai
import getpass
import tiktoken
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [3]:
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='e약은요.csv')
data = loader.load()

In [26]:
pages = loader.load_and_split()

In [6]:
pages[0].page_content

'\ufeff품목일련번호: 197900277\n제품명: 게보린정(수출명:돌로린정)\n업체명: 삼진제약(주)\n주성분: 아세트아미노펜,카페인무수물,이소프로필안티피린\n이 약의 효능은 무엇입니까?: 이 약은 두통, 치통, 발치(이를 뽑음)후 동통(통증), 인후(목구멍)통, 귀의 통증, 관절통, 신경통, 요(허리)통, 근육통, 견통(어깨통증), 타박통, 골절통, 염좌통(삔 통증), 월경통(생리통), 외상(상처)통의 진통과 오한(춥고 떨리는 증상), 발열시의 해열에 사용합니다.\n이 약은 어떻게 사용합니까?: 성인은 1회 1정 1일 3회까지 공복시를 피해 복용합니다.\n\n복용간격은 4시간 이상으로 합니다.\n\n이 약은 원칙적으로 단기간 복용합니다.\n이 약을 사용하기 전에 반드시 알아야 할 내용은 무엇입니가?: 매일 세잔 이상 정기적 음주자가 이 약 또는 다른 해열진통제를 복용할 때는 의사 또는 약사와 상의하십시오. 간손상을 일으킬 수 있습니다.\n\n아세트아미노펜으로 일일 최대 용량(4,000 mg)을 초과하여 복용하지 마십시오. 간손상을 일으킬 수 있습니다. 아세트아미노펜을 포함하는 다른 제품과 함께 복용하지 마십시오.\n이 약의 사용상 주의사항은 무엇입니까?: 이 약에 과민증 환자,\xa0다른 해열진통제(비스테로이드성 소염제), 감기약 복용시 천식발작 유발 또는 그 경험자,\xa0글루코스-6-인산 탈수소효소결핍증, 급성 간헐성(시간 간격을 두고 되풀이하여) 포르피린증, 과립백혈구감소증, 중증 간장애, 중증 신장애, 출혈 소인, 15세 미만의 소아, 소화성궤양, 심한 혈액 이상, 심한 심장기능저하,\xa0바르비탈계 약물, 삼환계 항우울제, 알코올을 복용한 환자는 이 약을\xa0복용하지 마십시오.\n\n이 약을 복용하기 전에 수두 또는 인플루엔자에 감염되어 있거나 또는 의심되는 15세 이상의 청소년, 갑상샘질환, 당뇨병, 고혈압, 몸이 약한 사람, 고열, 고령자, 임부 또는 임신하고 있을 가능성이 있는 여성, 수유부, 속쓰림, 위부불쾌감, 위통과 같은 위장문제가

In [27]:
print("data lenght : ",len(data))
print("data lenght : ",len(pages))

data lenght :  4650
data lenght :  4650


In [23]:
pages[0]

Document(page_content='\ufeff품목일련번호: 197900277\n제품명: 게보린정(수출명:돌로린정)\n업체명: 삼진제약(주)\n주성분: 아세트아미노펜,카페인무수물,이소프로필안티피린\n이 약의 효능은 무엇입니까?: 이 약은 두통, 치통, 발치(이를 뽑음)후 동통(통증), 인후(목구멍)통, 귀의 통증, 관절통, 신경통, 요(허리)통, 근육통, 견통(어깨통증), 타박통, 골절통, 염좌통(삔 통증), 월경통(생리통), 외상(상처)통의 진통과 오한(춥고 떨리는 증상), 발열시의 해열에 사용합니다.\n이 약은 어떻게 사용합니까?: 성인은 1회 1정 1일 3회까지 공복시를 피해 복용합니다.\n\n복용간격은 4시간 이상으로 합니다.\n\n이 약은 원칙적으로 단기간 복용합니다.\n이 약을 사용하기 전에 반드시 알아야 할 내용은 무엇입니가?: 매일 세잔 이상 정기적 음주자가 이 약 또는 다른 해열진통제를 복용할 때는 의사 또는 약사와 상의하십시오. 간손상을 일으킬 수 있습니다.\n\n아세트아미노펜으로 일일 최대 용량(4,000 mg)을 초과하여 복용하지 마십시오. 간손상을 일으킬 수 있습니다. 아세트아미노펜을 포함하는 다른 제품과 함께 복용하지 마십시오.\n이 약의 사용상 주의사항은 무엇입니까?: 이 약에 과민증 환자,\xa0다른 해열진통제(비스테로이드성 소염제), 감기약 복용시 천식발작 유발 또는 그 경험자,\xa0글루코스-6-인산 탈수소효소결핍증, 급성 간헐성(시간 간격을 두고 되풀이하여) 포르피린증, 과립백혈구감소증, 중증 간장애, 중증 신장애, 출혈 소인, 15세 미만의 소아, 소화성궤양, 심한 혈액 이상, 심한 심장기능저하,\xa0바르비탈계 약물, 삼환계 항우울제, 알코올을 복용한 환자는 이 약을\xa0복용하지 마십시오.\n\n이 약을 복용하기 전에 수두 또는 인플루엔자에 감염되어 있거나 또는 의심되는 15세 이상의 청소년, 갑상샘질환, 당뇨병, 고혈압, 몸이 약한 사람, 고열, 고령자, 임부 또는 임신하고 있을 가능성이 있는 여성, 수유부, 속쓰

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200, length_function=tiktoken_len)
texts = text_splitter.split_documents(pages)

In [8]:
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\Admin\anaconda3\envs\sba\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
#save to disk
emed_db = Chroma.from_documents(texts, hf,persist_directory="e약은요db")

In [ ]:
#벡터db의 데이터가 in-memory가 아니라 persistent storage인 disk에 저장되게 선언
emed_db.persist()
emed_db = None

In [10]:
# load from disk
emed_db = Chroma(persist_directory="e약은요db",embedding_function=hf)

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

openai = ChatOpenAI(model = "gpt-3.5-turbo",temperature=0)

retriever = emed_db.as_retriever(search_type="similarity", search_kwargs={'k':5})#search_type = "mmr",

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    openai, retriever, contextualize_q_prompt
)

#답변 생성
qa_system_prompt = """You are an assistant for question-answering tasks. \
ONLY USE the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(openai, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

# chat history 관리
store = {}

def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key= "input",
    history_messages_key= "chat_history",
    output_messages_key= "answer",
)


c:\Users\Admin\anaconda3\envs\sba\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [19]:
id = "0"
query1 = "게보린정 효능 알려줘"
conversational_rag_chain.invoke(
    {"input": query1},
    config={
        "configurable": {"session_id":id}
    },
)["answer"]

'게보린정은 통증 완화와 열 해소를 위해 사용되는 해열진통제입니다. 주로 두통, 치통, 근육통, 관절통, 생리통 등 다양한 통증을 완화하는 데 사용됩니다.'

In [20]:
query2 = "그럼 그 약 주의사항에 대해서 알려줘"

conversational_rag_chain.invoke(
    {"input": query2},
    config = {
        "configurable": {"session_id": id}
    },
)["answer"]

'게보린정을 복용하기 전에 신장이나 간장애, 노인, 간질 환자, 발작 위험성이 있는 환자, 소변 고임의 선행요인이 있는 환자는 의사나 약사와 상의해야 합니다. 또한 게보린정을 복용하는 동안 테오필린, 리토나비어, 과량의 알코올과 함께 복용하지 말아야 합니다.'

In [28]:
query3 = "이 약은 어떤 이상반응이 나타날 수 있습니까?"
conversational_rag_chain.invoke(
    {"input": query2},
    config = {
        "configurable": {"session_id": id}
    },
)["answer"]

'게보린정을 복용하기 전에 신장이나 간장애, 노인, 간질 환자, 발작 위험성이 있는 환자, 소변 고임의 선행요인이 있는 환자는 의사나 약사와 상의해야 합니다. 또한 게보린정을 복용하는 동안 테오필린, 리토나비어, 과량의 알코올과 함께 복용하지 말아야 합니다.'